### Combining both examples into one "Script"
We are going to make methods and take everything we wrote in the previous two chapters. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from outside_method import close_rate_bucket # see close_rate_agent_bucket

def create_output_from_call_center_data(call_center_data_frame):
    # create a copy so it's easier to re-run
    df = call_center_data_frame.copy()
    # create Close Rate Column 
    df['Close Rate'] = df['Sales']/df['Calls'] 
    # rename column 
    df = df.rename(columns={'Agent ID': 'agent_id'})
    # this is from outside method - See Import*
    df['close_rate_agent_buckets'] = df.apply(close_rate_bucket, axis=1) # see import to why this line works
    
    # buckets for call counts
    df['agent_call_buckets'] = 'Other' 
    # implement your logic
    df.loc[(df['Calls'] > 0) & (df['Calls'] < 50), 'agent_call_buckets'] = 'Low'
    df.loc[(df['Calls'] >= 50) & (df['Calls'] < 100), 'agent_call_buckets'] = 'Medium'
    df.loc[(df['Calls'] >= 100), 'agent_call_buckets'] = 'High'
    
    # create a group by
    group_by = df[['agent_id', 'agent_call_buckets', 'close_rate_agent_buckets']]
    group_by = group_by.groupby(['agent_call_buckets', 'close_rate_agent_buckets']).count().reset_index()
    group_by = group_by.rename(columns={'agent_id': 'number_of_agents'})
    
    # create a data frame with top performing agents
    great_agents = df[df['close_rate_agent_buckets'] == 'Great'] 
    
    # create a dataset with Poor performing Agents
    poor_agents = df[df['close_rate_agent_buckets'] == 'Poor'] 
    
    return group_by, great_agents, poor_agents

def create_stacked_bar_chart(df):
    df = df.copy()
    labels = ['Poor', 'Okay', 'Average', 'Good', 'Great']
    high_calls = [len(df[(df['close_rate_agent_buckets'] == 'Poor') & (df['agent_call_buckets'] == 'High')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Okay') & (df['agent_call_buckets'] == 'High')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Average') & (df['agent_call_buckets'] == 'High')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Good' )& (df['agent_call_buckets'] == 'High')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Great') & (df['agent_call_buckets'] == 'High')]),]

    medium_calls = [len(df[(df['close_rate_agent_buckets'] == 'Poor') & (df['agent_call_buckets'] == 'Medium')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Okay') & (df['agent_call_buckets'] == 'Medium')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Average') & (df['agent_call_buckets'] == 'Medium')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Good' )& (df['agent_call_buckets'] == 'Medium')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Great') & (df['agent_call_buckets'] == 'Medium')]),]

    low_calls = [len(df[(df['close_rate_agent_buckets'] == 'Poor') & (df['agent_call_buckets'] == 'Low')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Okay') & (df['agent_call_buckets'] == 'Low')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Average') & (df['agent_call_buckets'] == 'Low')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Good' )& (df['agent_call_buckets'] == 'Low')]),
                 len(df[(df['close_rate_agent_buckets'] == 'Great') & (df['agent_call_buckets'] == 'Low')]),]

    width = 0.35       # the width of the bars: can also be len(x) sequence

    fig, ax = plt.subplots()

    ax.bar(labels, high_calls, width, label='High Call Volume')
    ax.bar(labels, medium_calls, width, bottom=high_calls, label='Medium Call Volume')
    ax.bar(labels, low_calls, width, bottom=medium_calls, label='Low Call Volume')

    ax.set_ylabel('Number of Agents')
    ax.set_xlabel('Close Rate Bucket')
    ax.set_title('Agents by Call Count and Close Rate ')
    ax.legend()
    fig.savefig("score_card_chart.png")
    return print("completed score card chart")

def create_top_agent_chart(df):
    df = df.copy()
    fig, ax = plt.subplots()
    data = df['Close Rate'].tolist()
    labels = df['Agent Name'].tolist()
    plt.xticks(range(len(data)), labels, rotation = 45) # try to run this without 'rotation'
    plt.xlabel('Agent Name')
    plt.ylabel('Close Rate')
    plt.title('Top Performing Agents')
    plt.bar(range(len(data)), data) 
    plt.savefig("top_agent_chart.png")
    
    return print('top agent chart created')

def create_needs_imporvement_chart(df):
    df = df.copy()
    fig, ax = plt.subplots()
    data = df['Close Rate'].tolist()
    labels = df['Agent Name'].tolist()
    plt.xticks(range(len(data)), labels, rotation = 45) # try to run this without 'rotation'
    plt.xlabel('Agent Name')
    plt.ylabel('Close Rate')
    plt.title('Agents Agents that Need Improment')
    plt.bar(range(len(data)), data) 
    
    plt.savefig("needs_improvement_agent_chart.png")
    
    return print('needs imporvement done')

In [ ]:
df = pd.read_excel('Demo Data/Calls and Sales per Agent.xlsx')                  
group_by, great_agents, poor_agents = create_output_from_call_center_data(df)  # returns three data frames
create_stacked_bar_chart(group_by) # put each dataframe in the correct matplotlib method
create_top_agent_chart(great_agents)
create_needs_imporvement_chart(poor_agents)

### Send Email to your Boss
We can take everything we have done and send a summary email to the boss.

In [ ]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
import os
mail = outlook.CreateItem(0)
mail.To = 'john@refinere.com'
mail.CC = 'john@refinere.com'
mail.BCC = 'john@refinere.com'
mail.Subject = 'Weekly Call Center Data'
mail.Body = '''
        Hi Boss, 
        
        This is the weekly call center data excel document and charts. Please see attached. 
        
        Thanks, 
        John
'''
#mail.HTMLBody = '<h2>HTML Message body</h2>' #this field is optional

# To attach a file to the email (optional):
attachment  = os.getcwd() + '\Call Center Data.xlsx'
picture1 = os.getcwd() + "\score_card_chart.png"
picture2 = os.getcwd() + '\needs_imporvement_agent_chart.png'
picture3 = os.getcwd() + '\top_agent_chart.png'
mail.Attachments.Add(attachment)
mail.Attachments.Add(picture1)
mail.Attachments.Add(picture2)
mail.Attachments.Add(picture3)
mail.Send()